### **Código do `MVP` (prototipação)**

> **Contextualização**

O projeto em questão visa facilitar a criação de um gráfico de `Gantt` junto com um currículo em `LaTeX` salvo em `PDF`.

Sendo assim, teremos as respectivas etapas à serem desenvolvidas:

1. `Web Scrapping` das informações do próprio `LinkedIn`
2. Criação do gráfico de `Gantt`
3. Compilação dos arquivos em `LaTeX` para `PDF`.

> **Web Scraping**

* **Bibliotecas**:

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import os
from dotenv import load_dotenv


* **Configurações:**

In [2]:
# Pasta dos códigos
path_codigos = os.path.dirname(os.getcwd())

# Pasta das entradas
path_entradas = os.path.join(path_codigos, 'Entrada')

# Apontamento para a pasta de entrada
os.chdir(path_entradas)

# Carregando as variáveis de ambiente
load_dotenv('credenciais.env')

# Dicionário para as informações coletadas
resultados = {}

# # Selenium de forma oculta
# configuracoes = webdriver.ChromeOptions()
# configuracoes.add_argument("--headless=new")


* **Links:**

In [3]:
# Usuário desejado
nome_perfil = os.environ['perfil']

# URL padrão
url_linkedin = 'https://www.linkedin.com/'

# Link direto para o perfil
url_perfil = url_linkedin + 'in' + f'/{nome_perfil}/'

# Página de Login
url_login = url_linkedin + '/login/pt'

# Detalhes
url_detalhes = url_perfil + 'details/'

# Seções
url_contatos = url_perfil + '/overlay/contact-info/'
url_experiencias = url_detalhes + 'experience'
url_estudos = url_detalhes + 'education'
url_certificados = url_detalhes + 'certifications'
url_voluntarios = url_detalhes + 'volunteering-experiences'

* **Navegador:**

In [4]:
# Criando o navegador
navegador = webdriver.Chrome()  # options=configuracoes

# Adicionando delay de carregamento
time.sleep(1)


* **Login:**

In [5]:
# Acessando o local de login
navegador.get(url_login)

# TODO: Remover sleeps e, adicionar algum wait da página carregada

# Adicionando delay de carregamento
time.sleep(1)

# Preenchendo o email
navegador.find_element(By.ID, 'username').send_keys(
    os.environ['usuario']
)

# Preenchendo a senha
navegador.find_element(By.ID, 'password').send_keys(
    os.environ['senha']
)

# Entrando
navegador.find_element(By.ID, 'password').send_keys(Keys.RETURN)

# Adicionando delay devido aprovação de login
time.sleep(10)


* **Perfil**:

In [6]:
def get_nome(html):
    '''
    Coleta o nome do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Nome do perfil
    nome = html.find('h1').text

    # Removendo espaçamentos exagerados
    nome = nome.strip()

    # Retornando o nome do profissional
    return nome


In [7]:
def get_frase(html):
    '''
    Coleta a frase de efeito do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Frase de efeito
    frase = html.find('div', {'class': 'text-body-medium'}).text

    # Removendo espaçamentos exagerados
    frase = frase.strip()

    # Retornando a frase de efeito
    return frase


In [8]:
def get_sobre(html):
    '''
    Coleta o sobre do profissional por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Filtro dos spans
    spans = html.find_all('span')

    # Variável auxiliar para contador de, onde está o sobre
    index_aux = 0

    # Iterando a lista dos spans até encontrar qual o index está o Sobre
    for index, div in enumerate(spans):

        # Se encontrar o sobre
        if 'Sobre' in div:

            # Atribui o índice
            indice = index

    # Pegando o Sobre
    sobre = spans[indice + 2].text

    # Removendo espaçamentos exagerados
    sobre = sobre.strip()

    # Retornando o sobre
    return sobre

In [9]:
# Acessando o perfil do usuários
navegador.get(url=url_perfil)

# Adicionando delay de carregamento
time.sleep(3)

# Pegando o conteúdo HTML do perfil
html_perfil = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

# Atribuindo os resultados coletados
resultados['perfil'] = {
    'nome': get_nome(html_perfil),
    'frase': get_frase(html_perfil),
    'sobre': get_sobre(html_perfil),
    # 'link': url_perfil
}

# Informativo
display(resultados['perfil'])


{'nome': 'Murilo Chaves Jayme',
 'frase': 'Cientista de dados com foco em mercado financeiro',
 'sobre': 'Bacharel em Sistemas de Informação pela ULBRA e atualmente cursando dois programas de pós-graduação: MBA em Big Data Science pela FIAP e MBA em Ciência de Dados para o Mercado Financeiro na XP Educação. Tem experiência como pesquisador em projetos financiados pela FAPEG. E, atualmente buscando oportunidades (Cientista de dados ou Analista Quantitativo) para atuação com pesquisa e inovação voltadas para o mercado financeiro.'}

* **Contatos:**

In [10]:
def get_contatos(html):
    '''
    Coleta as informações de contato por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Filtro da seção de contato
    secoes = html.find_all('section', {'class': 'pv-contact-info__contact-type'})

    # Variável auxiliar de tratamento das strings
    tratamentos = {
        'Seu perfil': 'perfil',
        'sites': 'sites',
        'telefone': 'telefone',
        'Endereço': 'endereco',
        'E-mail': 'email',
        'Data de nascimento': 'data_nascimento',
        '(Pessoal)': 'pessoal',
        '(Portfólio)': 'portfolio',
        '(Celular)': 'celular',
    }

    # Variável auxiliar dos resultados
    resultados = {}

    # Variável auxiliar de processamento de links
    processar_links = False

    # Para cada seção
    for secao in secoes:

        # Capturando o título
        titulo = secao.find('h3').text.strip()

        # Se, existir um título
        if titulo:

            # Realizando tratamentos
            titulo = tratamentos[titulo]

            # Adicionando o título aos resultados
            resultados[titulo] = ''

        # Se existir links
        try:
            # Coletando possíveis links
            links = secao.find_all('a')

            # Extraindo os links
            links = [link.text.strip() for link in links]


            # Se, for apenas 1 link
            if len(links) == 1:

                # Adicionando o valor do link
                resultados[titulo] = links[0]
            
            # Se existir mais de 1 link
            elif len(links) > 1:

                # Atribuindo variável para escopo
                processar_links = True

        except:
            pass
        
        # Se existir conteúdos
        try:
            # Pegando os spans
            spans = secao.find_all('span')

            # Separando os textos dos spans
            spans = [span.text.strip() for span in spans]

            # Se, existir conteúdos
            if len(spans) == 1:

                # Adicionando os spans
                resultados[titulo] = spans[0]
            
            # Se, existir links e, o span for maior
            elif len(spans) > 1:

                # Se for os sites
                if 'sites' == titulo:

                    # Adicionando a seção dos sites
                    resultados[titulo] = {}

                    # Para cada span
                    for index, site in enumerate(spans):

                        # Adicionando nos resultados
                        resultados[titulo][tratamentos[site]] = links[index]

                # Se for o telefone
                elif 'telefone' in titulo:

                    # Adicionando a seção
                    resultados[titulo] = {}

                    # Adicionando os reultados
                    resultados[titulo][tratamentos[spans[-1]]] = spans[0]

        except:
            pass

    # Retornando os resultados
    return resultados


In [11]:
import re  # Expressões regulares

def get_endereco(endereco: str) -> dict:
    '''
    Realiza a separação dos dados do endereço em uma forma mais estruturada desde que, o endereço esteja no padrão do Google Maps.
    '''

    # Tipo do endereço
    tipo = endereco.split()[0]

    # Logradouro
    logradouro = ' '.join(endereco.split(', ')[0].split()[1:])

    # Número
    numero = endereco.split(', ')[1].split()[0]

    # Evitar erro caso, não exista o complemento
    try:

        # Complemento
        complemento = re.search(r'\((.*?)\)', endereco).group(1)

    except:

        # Valor nulo
        complemento = ''

    # Bairro
    bairro = endereco.split(' - ')[1].split(', ')[0]

    # Cidade
    cidade = endereco.split(' - ')[1].split(', ')[1]

    # Estado
    estado = endereco.split(' - ')[2].split(', ')[0]

    # CEP
    cep = endereco.split(' - ')[2].split(', ')[1]

    # Construindo os resultados
    resultados = {
        'tipo': tipo,
        'logradouro': logradouro,
        'numero': numero,
        'complemento': complemento,
        'bairro': bairro,
        'cidade': cidade,
        'estado': estado,
        'cep': cep,
    }

    # Retornadno os resultados
    return resultados


In [12]:
# Acessando a página de contatos
navegador.get(url=url_contatos)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
html_contatos = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

# Atribuindo os resultados coletados
resultados['contatos'] = get_contatos(html_contatos)

# Padronizando o endereço
resultados['contatos']['endereco'] = get_endereco(resultados['contatos']['endereco'])

# Informativo
display(resultados['contatos'])

{'perfil': 'linkedin.com/in/murilochaves',
 'sites': {'pessoal': 'murilochaves.dev/',
  'portfolio': 'github.com/murilochaves'},
 'telefone': {'celular': '+55 (34) 99864-2301'},
 'endereco': {'tipo': 'R.',
  'logradouro': 'Vinte e Nove de Outubro',
  'numero': '701',
  'complemento': '',
  'bairro': 'Patrimônio',
  'cidade': 'Uberlândia',
  'estado': 'MG',
  'cep': '38411-068'},
 'email': 'murilochavesj@gmail.com',
 'data_nascimento': '23 de janeiro'}

* **Experiência:**

In [168]:
# Acessando a página de experiências profissionais
navegador.get(url=url_experiencias)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
html_experiencias = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

In [197]:
def get_experiencias(html):
    '''
    Coleta as informações das experiências profissionais por meio de um HTML da página do perfil do LinkedIn.
    '''

    # Filtro dos spans
    spans = html.find_all('span', {'class': 'visually-hidden'})

    # Variável auxiliar para os textos extraídos
    experiencias = []

    # Percorrendo os spans
    for span in spans:

        # Verificando se, encontramos o padrão que, reflete os conteúdos sem ruídos
        if '<!-- -->' in str(span):

            # Adicionando o valor de extração dos textos
            experiencias.append(span.text.strip())

    # Variável auxiliar indicativo de empresa
    empresa_aux = False

    # Variável auxiliar dos meses
    meses = [
        'jan', 'fev', 'mar', 
        'abr', 'mai', 'jun', 
        'jul', 'ago', 'set',
        'out', 'nov', 'dez',
    ]

    # Variável auxiliar de ano ou mes
    indicativo_tempo = ['a', 'm']

    # Variável auxiliar de experiências compostas
    experiencia_individual, experiencia_composta = False, False

    # Variáveis auxiliares de índices compostos
    indice_inicio_composto, indice_composto_experiencia = 0, 0
    indice_inicio_individual, indice_individual_experiencia = 0, 0

    # Resultados da coleta
    resultados = {}

    # Percorrendo as extrações
    for index, experiencia in enumerate(experiencias):

        # Evitando erro de indexação
        try:

            # Pegando próxima experiência
            proxima_experiencia = experiencias[index + 1]

            # Vetorizando
            proxima_experiencia = proxima_experiencia.split()

            # Verificando indicador de meses
            if len(proxima_experiencia[1]) == 1 and proxima_experiencia[1] in indicativo_tempo:

                # Informativo
                # print('\n#-#-#   1. COMPOSTO   #-#-#')

                # Auxiliares de escopo
                experiencia_individual, experiencia_composta = False, True

                # Auxiliar do índice composto
                indice_inicio_composto = index

                # Atribuindo empresa da experiência composta
                resultados[experiencia] = []

                # Empresa
                empresa = experiencia

                # Localização
                localizacao = experiencias[index + 2]

                # Tempo total
                tempo_total = experiencias[index + 1]

        except:
            pass

        # Separando as outras experiências
        try:

            # Pegando 2 experiências abaixo
            experiencia_retroativa = experiencias[index - 2]

            # Experiências futuras
            experiencia_futura = experiencias[index + 2]

            # Vetorizando
            experiencia_retroativa = experiencia_retroativa.split()
            experiencia_futura = experiencia_futura.split()

            # Se, for indicativo dos meses
            if experiencia_retroativa[0] in meses and experiencia_futura[0] in meses:

                # Informativo
                # print('\n*-*-*   INDIVIDUAL   *-*-*')

                # Auxiliares de escopo
                experiencia_individual, experiencia_composta = True, False

                # Auxiliar do índice individual
                indice_individual_experiencia = index

        except:
            pass

        # Se for uma experiência composta
        if experiencia_composta and index == indice_inicio_composto + 3:

            # Auxiliar do índice composto da experiência
            indice_composto_experiencia = index

            # Informativo
            # print('\n;-;-;   1. EXPERIÊNCIA   ;-;-;')

            # Adicionando a posição
            resultados[empresa].append({
                'posicao': experiencia,
                'periodo': experiencias[index + 1],
                'descricao': experiencias[index + 2],
            })

        # Iterando a estrutura da experiência composta de 3 em 3 campos
        if experiencia_composta and index > indice_composto_experiencia and index - indice_composto_experiencia == 3:

            # Informativo
            # print('\n;-;-;   2. EXPERIÊNCIA   ;-;-;')

            # Auxiliar do índice composto da experiência
            indice_composto_experiencia = index

            # Adicionando a posição
            resultados[empresa].append({
                'posicao': experiencia,
                'periodo': experiencias[index + 1],
                'localizacao': localizacao,
                'descricao': experiencias[index + 2],
            })

        # Iterando a estrutura da experiência individual de
        if experiencia_individual and index == indice_individual_experiencia:

            # Informativo
            # print('\n;-;-;   3. EXPERIÊNCIA   ;-;-;')

            # Auxiliar do índice individual da experiência
            indice_individual_experiencia = index

            # Adicionando a empresa
            resultados[experiencias[index + 1]] = {}

            # Adicionando a posição
            resultados[experiencias[index + 1]]['posicao'] = experiencia
            resultados[experiencias[index + 1]]['periodo'] = experiencias[index + 2]
            resultados[experiencias[index + 1]]['localizacao'] = experiencias[index + 3]
            resultados[experiencias[index + 1]]['descricao'] = experiencias[index + 4]

        # Iterando a estrutura da experiência composta de 3 em 3 campos
        if experiencia_individual and index > indice_individual_experiencia and index - indice_individual_experiencia == 5:

            # Informativo
            # print('\n;-;-;   4. EXPERIÊNCIA   ;-;-;')

            # Auxiliar do índice composto da experiência
            indice_individual_experiencia = index

            # Adicionando a empresa
            resultados[experiencias[index + 1]] = {}

            # Adicionando a posição
            resultados[experiencias[index + 1]]['posicao'] = experiencia
            resultados[experiencias[index + 1]]['periodo'] = experiencias[index + 2]
            resultados[experiencias[index + 1]]['localizacao'] = experiencias[index + 3]
            resultados[experiencias[index + 1]]['descricao'] = experiencias[index + 4]
        
        # print(index, experiencia)

    return resultados

get_experiencias(html_experiencias)


{'Callink': [{'posicao': 'Líder técnico',
   'periodo': 'set de 2022 - jul de 2023 · 11 meses',
   'descricao': 'Como Líder Técnico no Squad Quant (equipe de finanças quantitativas), minha responsabilidade era garantir a qualidade dos produtos desenvolvidos para o mercado financeiro. Assumi o papel de auxiliar na estratégia, prototipação, execução, entrega e manutenção dos projetos. Sempre me atualizado com as tecnologias e tendências em Data Science aplicadas às finanças, a fim de aprimorar continuamente os processos.'},
  {'posicao': 'Cientista de dados pleno',
   'periodo': 'jun de 2022 - jul de 2023 · 1 ano 2 meses',
   'localizacao': 'Uberlândia, Minas Gerais, Brasil',
   'descricao': 'Estive envolvido no desenvolvimento e comunicação dos resultados de projetos como a análise de CPC (Contato com a Pessoa Certa), People Analytics (com o objetivo de compreender e reduzir os desligamentos na empresa) e a detecção de fraudes em atestados. Interagi diretamente com os setores de PCP, RH

* **Formação acadêmica:**

In [15]:
# Acessando a página dos estudos
navegador.get(url=url_estudos)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
estudos = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

* **Licenças e Certificados:**

In [16]:
# Acessando a página dos certificados
navegador.get(url=url_certificados)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
certificados = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)

* **Trabalho voluntário:**

In [17]:
# Acessando a página dos voluntariados
navegador.get(url=url_voluntarios)

# Adicionando tempo para renderização da página
time.sleep(3)

# Conteúdo das experiências profissionais
voluntarios = BeautifulSoup(
    navegador.page_source,
    'html.parser'
)